In [ ]:
# queue status
!sinfo --long

In [ ]:
# node status
!sinfo --long -N

In [ ]:
# CPU queue job list
!squeue -p cpu --long

In [ ]:
# prepare python env for jobs
!sbatch --partition=cpu --time=0:10:00 --job-name=prepare-env --ntasks=2 run-hydra-env.sh

In [ ]:
# prepare python env for jobs
!sbatch --partition=tut --time=0:10:00 --job-name=prepare-env --ntasks=1 run-hydra-env.sh

In [ ]:
%%bash
sbatch --partition=cpu \
--ntasks=40 --ntasks-per-node=20 --time=24:00:00 \
--wrap 'n_workers=40 min_budget=50 max_budget=600 n_iterations=100 hopfield_steps=100 srun ./hydra-task.sh' \
--job-name=optimize --cpus-per-task=1 --mem-per-cpu=6000 --no-requeue \
--dependency=afterok:525446

In [ ]:
%%bash
sbatch --partition=cpu \
--ntasks=40 --ntasks-per-node=40 --time=4:00:00 \
--wrap 'n_workers=40 min_budget=200 max_budget=1200 n_iterations=100 hopfield_steps=100 dataset=BMaN srun ./hydra-task.sh' \
--job-name=optimize --cpus-per-task=1 --mem-per-cpu=3000 --no-requeue \
--dependency=afterok:526834

In [ ]:
%%bash
sbatch --partition=tut \
--ntasks=48 --ntasks-per-node=48 --time=2:00:00 \
--wrap 'n_workers=48 min_budget=200 max_budget=1200 n_iterations=100 hopfield_steps=100 srun ./hydra-task.sh' \
--job-name=optimize --cpus-per-task=1 --mem-per-cpu=4000 --no-requeue \
--dependency=afterok:526418

In [ ]:
# CPU queue job list
!squeue -p cpu --long
# our job status
!squeue --user $USER --long

In [ ]:
# !scancel 524099

In [ ]:
ls slurm-*.out

In [ ]:
ls ./workdir/

In [ ]:
# when did the job start and end?
!cat slurm-526835.out | grep 2021

**Don't forget to run hydra-setup.ipynb**

In [ ]:
import pickle
from matplotlib import pyplot as plt

import hpbandster.visualization as hpvis

with open('workdir/526835.pkl', 'rb') as fh:
    result = pickle.load(fh)

result

In [ ]:
all_runs = result.get_all_runs()
print(len(all_runs))
all_runs[-1]

In [ ]:
# get the 'dict' that translates config ids to the actual configurations
id2conf = result.get_id2config_mapping()
# id2conf[(0,0,0)]

In [ ]:
# Here is how you get he incumbent (best configuration)
inc_id = result.get_incumbent_id()
inc_id

In [ ]:
# let's grab the run on the highest budget
inc_runs = result.get_runs_by_id(inc_id)
inc_run = inc_runs[-1]
inc_runs

In [ ]:
# We have access to all information: the config, the loss observed during
#optimization, and all the additional information
inc_loss = inc_run.loss
inc_config = id2conf[inc_id]['config']
# inc_test_loss = inc_run.info['test_loss']

print('Best found configuration:')
print(inc_config)
# print('It achieved accuracies of %f (train) and %f (test).'%(1-inc_loss, 1-inc_test_loss))

In [ ]:
# Let's plot the observed losses grouped by budget,
hpvis.losses_over_time(all_runs)

In [ ]:
# the number of concurent runs,
hpvis.concurrent_runs_over_time(all_runs)

In [ ]:
# and the number of finished runs.
hpvis.finished_runs_over_time(all_runs)

In [ ]:
# This one visualizes the spearman rank correlation coefficients of the losses
# between different budgets.
hpvis.correlation_across_budgets(result)

In [ ]:
# For model based optimizers, one might wonder how much the model actually helped.
# The next plot compares the performance of configs picked by the model vs. random ones
hpvis.performance_histogram_model_vs_random(all_runs, id2conf)